# GPU
:label:`sec_use_gpu`

在:numref:`tab_intro_decade`中，我们展示了过去二十年计算能力的快速增长。简而言之，自2000年以来，GPU性能每十年增长1000倍。这提供了巨大的机会，但也表明了对这种性能有显著的需求。

在本节中，我们将开始讨论如何利用这种计算性能来进行你的研究。首先使用单个GPU，之后再讨论如何使用多个GPU和多台服务器（配备多个GPU）。

具体来说，我们将讨论如何使用单个NVIDIA GPU进行计算。首先，确保至少安装了一个NVIDIA GPU。然后，下载[NVIDIA驱动程序和CUDA](https://developer.nvidia.com/cuda-downloads)，并按照提示设置适当的路径。一旦这些准备工作完成，可以使用`nvidia-smi`命令来（**查看显卡信息**）。

在PyTorch中，每个数组都有一个设备；我们经常将其称为*上下文*。到目前为止，默认情况下，所有变量及其相关计算都被分配到了CPU上。通常，其他上下文可能是各种GPU。当我们跨多台服务器部署任务时，事情可能会变得更加复杂。通过智能地将数组分配给上下文，我们可以最小化设备之间传输数据所花费的时间。例如，在具有GPU的服务器上训练神经网络时，我们通常希望模型的参数位于GPU上。

要运行本节中的程序，
您至少需要两个GPU。
请注意，这对于大多数台式计算机来说可能过于奢侈，
但在云中很容易获得，例如，
通过使用AWS EC2多GPU实例。
几乎所有其他部分都*不需要*多个GPU，但在这里我们只是想说明不同设备之间的数据流。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

## [**计算设备**]

我们可以指定诸如CPU和GPU之类的设备，
用于存储和计算。
默认情况下，张量在主内存中创建，
然后使用CPU进行计算。

在 PyTorch 中，CPU 和 GPU 可以用 `torch.device('cpu')` 和 `torch.device('cuda')` 表示。
需要注意的是，`cpu` 设备
意味着所有物理 CPU 和内存。
这意味着 PyTorch 的计算
将尝试使用所有 CPU 核心。
然而，`gpu` 设备仅表示一张卡
及其对应的内存。
如果有多个 GPU，我们使用 `torch.device(f'cuda:{i}')`
来表示第 $i$ 个 GPU（$i$ 从 0 开始）。
另外，`gpu:0` 和 `gpu` 是等价的。

In [2]:
def cpu():  #@save
    """Get the CPU device."""
    return torch.device('cpu')

def gpu(i=0):  #@save
    """Get a GPU device."""
    return torch.device(f'cuda:{i}')

cpu(), gpu(), gpu(1)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))

我们可以（查询可用的GPU数量。）

In [3]:
def num_gpus():  #@save
    """Get the number of available GPUs."""
    return torch.cuda.device_count()

num_gpus()

2

现在我们[**定义两个方便的函数，即使请求的GPU不存在，也能让我们运行代码。**]

In [4]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if num_gpus() >= i + 1:
        return gpu(i)
    return cpu()

def try_all_gpus():  #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    return [gpu(i) for i in range(num_gpus())]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0), device(type='cuda', index=1)])

## 张量和GPU

默认情况下，张量是在CPU上创建的。
我们可以[**查询张量所在的设备。**]

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

需要注意的是，每当我们要对多个项进行操作时，它们需要位于同一设备上。例如，如果我们对两个张量求和，我们需要确保两个参数都位于同一设备上——否则框架将不知道在哪里存储结果，甚至不知道如何决定在哪里执行计算。

### GPU上的存储

有几种方法可以[**将张量存储在GPU上**]。例如，我们可以在创建张量时指定存储设备。接下来，我们在第一个`gpu`上创建张量变量`X`。在GPU上创建的张量只消耗该GPU的内存。我们可以使用`nvidia-smi`命令来查看GPU的内存使用情况。通常，我们需要确保不会创建超出GPU内存限制的数据。

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

假设你至少有两个GPU，以下代码将（**在第二个GPU上创建一个随机张量 `Y`。**）

In [7]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.0022, 0.5723, 0.2890],
        [0.1456, 0.3537, 0.7359]], device='cuda:1')

### 复制

[**如果我们想要计算 `X + Y`，
我们需要决定在哪里执行这个操作。**]
例如，如 :numref:`fig_copyto` 所示，
我们可以将 `X` 转移到第二个 GPU
并在那里执行操作。
*不要*简单地将 `X` 和 `Y` 相加，
因为这会导致异常。
运行时引擎将不知道如何处理：
它无法在同一设备上找到数据，因此会失败。
由于 `Y` 存储在第二个 GPU 上，
我们需要先将 `X` 移动到那里才能进行相加。

![Copy data to perform an operation on the same device.](../img/copyto.svg)
:label:`fig_copyto`

In [8]:
Z = X.cuda(1)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')


现在[**数据（包括`Z`和`Y`）都在同一GPU上，我们可以将它们相加。**]

In [9]:
Y + Z

tensor([[1.0022, 1.5723, 1.2890],
        [1.1456, 1.3537, 1.7359]], device='cuda:1')

但是如果变量`Z`已经存在于你的第二个GPU上呢？
如果我们仍然调用`Z.cuda(1)`会发生什么？
它会返回`Z`，而不会创建副本或分配新的内存。

In [10]:
Z.cuda(1) is Z

True

### 旁注

人们使用GPU进行机器学习
是因为他们期望它们速度快。
但是在设备之间传输变量很慢：比计算慢得多。
所以我们希望你100%确定
在我们允许你这样做之前，你确实想要执行一个缓慢的操作。
如果深度学习框架只是自动进行了复制
而没有崩溃，那么你可能不会意识到
你写了一些效率低下的代码。

数据传输不仅慢，还使得并行化变得更加困难，
因为我们必须等待数据被发送（或者更准确地说是被接收）
才能继续进行更多的操作。
这就是为什么复制操作需要非常小心。
作为经验法则，许多小操作
比一个大操作糟糕得多。
此外，同时进行多个操作
比在代码中分散的多个单个操作要好得多，
除非你知道自己在做什么。
这是因为这种操作可能会阻塞，如果一个设备
必须等待另一个设备完成操作后才能做其他事情。
这有点像排队点咖啡
而不是通过电话预先下单
然后发现当你到达时咖啡已经准备好了。

最后，当我们打印张量或将张量转换为NumPy格式时，
如果数据不在主内存中，
框架会先将其复制到主内存中，
从而导致额外的传输开销。
更糟的是，它现在受制于可怕的全局解释器锁
这使得所有事情都得等待Python完成。


## [**神经网络和GPU**]

类似地，神经网络模型可以指定设备。
以下代码将模型参数放在GPU上。

In [11]:
net = nn.Sequential(nn.LazyLinear(1))
net = net.to(device=try_gpu())

在接下来的章节中，我们将看到更多关于如何在GPU上运行模型的例子，仅仅因为这些模型的计算量会稍微增加。

例如，当输入是GPU上的张量时，模型将在同一个GPU上计算结果。

In [12]:
net(X)

tensor([[0.7802],
        [0.7802]], device='cuda:0', grad_fn=<AddmmBackward0>)

让我们（**确认模型参数存储在同一GPU上。**）

In [13]:
net[0].weight.data.device

device(type='cuda', index=0)

让训练器支持GPU。

In [14]:
@d2l.add_to_class(d2l.Trainer)  #@save
def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
    self.save_hyperparameters()
    self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]

@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_batch(self, batch):
    if self.gpus:
        batch = [a.to(self.gpus[0]) for a in batch]
    return batch

@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_model(self, model):
    model.trainer = self
    model.board.xlim = [0, self.max_epochs]
    if self.gpus:
        model.to(self.gpus[0])
    self.model = model

简而言之，只要所有数据和参数都在同一设备上，我们就可以高效地学习模型。在接下来的章节中，我们将看到几个这样的例子。

## 总结

我们可以指定用于存储和计算的设备，例如CPU或GPU。
  默认情况下，数据是在主内存中创建
  然后使用CPU进行计算。
深度学习框架要求所有用于计算的输入数据
  都必须在同一设备上，
  无论是CPU还是同一个GPU。
不加注意地移动数据可能会导致性能显著下降。
  一个典型的错误如下：在GPU上为每个小批量计算损失
  并通过命令行（或记录到NumPy `ndarray`中）将其报告给用户
  这将触发全局解释器锁，从而阻塞所有GPU。
  更好的做法是分配
  GPU内的日志内存，并仅移动较大的日志。

## 练习

1. 尝试更大的计算任务，比如大矩阵的乘法，
   并观察CPU和GPU之间的速度差异。
   如果是一个计算量很小的任务呢？
1. 我们应该如何在GPU上读写模型参数？
1. 测量计算1000次
   $100 \times 100$矩阵-矩阵乘法所需的时间
   并每次记录输出矩阵的Frobenius范数。与在GPU上保持日志并将最终结果转移进行比较。
1. 测量同时在两个GPU上执行两次矩阵-矩阵乘法所需的时间。
   与在一个GPU上顺序计算进行比较。提示：你应该能看到几乎线性的扩展。

[讨论](https://discuss.d2l.ai/t/63)